# MobiML FL demo

Using Flower and MobiML




In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import pickle
import warnings
import pandas as pd
import geopandas as gpd
import numpy as np
from datetime import datetime, timedelta
from copy import deepcopy
from typing import Dict, List, Tuple
from pathlib import Path
from sklearn.metrics import log_loss
from sklearn.preprocessing import MultiLabelBinarizer
import pymeos

import flwr as fl
from flwr.common import Metrics, Context

import sys
sys.path.append("..")
from mobiml.datasets import AISDK, MOVER_ID, SHIPTYPE
from mobiml.transforms import TripExtractor, TrajectoryAggregator
from mobiml.preprocessing import StationaryClientExtractor, MobileClientExtractor
from mobiml.models import SummarizedAISTrajectoryClassifier
from mobiml.models.ais_trajectory_classifier import AISLoader, get_evaluate_fn, fit_round, weighted_average
from mobiml.utils import convert_wgs_to_utm


## Set FL simulation output verbosity settings


RAY_BACKEND_LOG_LEVEL: to declutter log output in shells, a low-volume log level has been chosen from [the source code](https://github.com/ray-project/ray/blob/master/src/ray/util/logging.cc#L273)

RAY_DEDUP_LOGS: to see logs from all clients instead of just one

In [3]:
os.environ['RAY_DEDUP_LOGS'] = '0'
os.environ['RAY_BACKEND_LOG_LEVEL'] = 'fatal'


In [4]:
#path = "./data/aisdk-2018-02.zip" # For a real-world experience, download AISDK input file (1 month of data, 14G) from here http://web.ais.dk/aisdata/aisdk-2018-02.zip

path = "./data/aisdk-4days.zip"

## Extract stationary client (antenna) data

In [5]:
antennas = ['Point (11.96524 57.70730)', 'Point (11.63979 57.71941)', 'Point (11.78460 57.57255)']
antenna_radius_meters = 25000


In [6]:
epsg_code = convert_wgs_to_utm(11.96524, 57.70730)

ids =  [{'client': i} for i in range(len(antennas))]
df = pd.DataFrame(ids)
df['geometry'] = gpd.GeoSeries.from_wkt(antennas)
gdf = gpd.GeoDataFrame(df, geometry=df.geometry, crs=4326)
gdf = gdf.to_crs(epsg_code)
gdf['geometry'] = gdf.buffer(antenna_radius_meters)

buffered_antennas =  gdf.to_crs(4326)
min_lon, min_lat, max_lon, max_lat = buffered_antennas.geometry.total_bounds

In [7]:
out_dir = "temp"
os.path.dirname(out_dir)
if not os.path.exists(out_dir):
    print(f"{datetime.now()} Creating output directory {out_dir} ...")
    os.makedirs(out_dir)

In [8]:
print(f"{datetime.now()} Loading data from {path}")
aisdk = AISDK(path, min_lon, min_lat, max_lon, max_lat)

2024-09-12 13:00:49.824616 Loading data from ./data/aisdk-4days.zip
2024-09-12 13:00:49.825432 Loading aisdk_20180201.csv ...
2024-09-12 13:01:09.195945 Loading aisdk_20180202.csv ...
2024-09-12 13:01:28.079027 Loading aisdk_20180203.csv ...
2024-09-12 13:01:46.741417 Loading aisdk_20180204.csv ...
2024-09-12 13:02:06.298207 Loaded Dataframe with 1157550 rows.


In [9]:
print(f"{datetime.now()} Extracting client data ...")
antenna_gdf = StationaryClientExtractor(aisdk).extract(buffered_antennas)

2024-09-12 13:02:06.320419 Extracting client data ...
2024-09-12 13:02:06.320502 Converting to GeoDataFrame ...
2024-09-12 13:02:27.689974 Computing overlay ...


In [10]:
stationary_feather_path = f"{out_dir}/ais-antenna.feather"


In [11]:
print(f"{datetime.now()} Writing output to {stationary_feather_path}")
antenna_gdf.to_feather(stationary_feather_path)

2024-09-12 13:02:37.562985 Writing output to temp/ais-antenna.feather


## Extract mobile client (vessel) data

In [12]:
ship_type = 'Towing' 
antenna_radius_meters = 25000  
bbox = [57.273, 11.196, 57.998, 12.223]  
min_lat, min_lon, max_lat, max_lon = bbox

In [13]:
mobile_feather_path = f"{out_dir}/ais-vessels.feather"

In [14]:
print(f"{datetime.now()} Loading data from {path}")
aisdk = AISDK(path, min_lon, min_lat, max_lon, max_lat)
vessels = deepcopy(aisdk)   # AISDK(path, min_lon, min_lat, max_lon, max_lat, vessel_type)
vessels.df = vessels.df[vessels.df.ship_type == ship_type]

2024-09-12 13:03:04.038456 Loading data from ./data/aisdk-4days.zip
2024-09-12 13:03:04.038975 Loading aisdk_20180201.csv ...
2024-09-12 13:03:21.819907 Loading aisdk_20180202.csv ...
2024-09-12 13:03:40.420676 Loading aisdk_20180203.csv ...
2024-09-12 13:03:59.297689 Loading aisdk_20180204.csv ...
2024-09-12 13:04:18.323894 Loaded Dataframe with 1297669 rows.


In [15]:
print(f"{datetime.now()} Extracting client data ...")
vessel_gdf = MobileClientExtractor(aisdk).extract(vessels, antenna_radius_meters)

2024-09-12 13:04:18.437274 Extracting client data ...
2024-09-12 13:04:18.444314 Creating PyMEOS points ...
2024-09-12 13:05:41.785502 Creating client trajectories ...
2024-09-12 13:05:41.864916 Creating PyMEOS trajectories ...
2024-09-12 13:05:41.883929 Calculating spatiotemporal intersections ...
1/2: 219012959
2/2: 236111925


In [16]:
print(f"{datetime.now()} Writing output to {mobile_feather_path}")
vessel_gdf.to_feather(mobile_feather_path)

2024-09-12 13:06:16.383659 Writing output to temp/ais-vessels.feather


## Prepare stationary and mobile client training data

In [17]:
h3_resolution = 8

stationary_training_file = f"temp/training-data-stationary.pickle"
mobile_training_file = f"temp/training-data-mobile.pickle"

In [18]:
def prepare_training_data(client_feather, outfile):
    print(f"{datetime.now()} Loading data from {client_feather} ...")
    gdf = gpd.read_feather(client_feather)
    vessels = gdf.groupby(MOVER_ID)[["ship_type", "Name"]].agg(pd.Series.mode)

    print(f"{datetime.now()} Extracting trips ...")
    trajs = TripExtractor(gdf).get_trips( gap_duration=timedelta(minutes=60))  

    print(f"{datetime.now()} Computing trajectory features ...")
    t_df = TrajectoryAggregator(trajs, vessels).aggregate_trajs(h3_resolution)

    with open(outfile.replace("training-data", "vessel"), "wb") as out_file:
        pickle.dump(vessels, out_file)

    with open(outfile, "wb") as out_file:
        pickle.dump(t_df, out_file)
    print(f"{datetime.now()} training data written to {outfile}")


In [19]:
prepare_training_data(stationary_feather_path, stationary_training_file)


2024-09-12 13:09:56.139069 Loading data from temp/ais-antenna.feather ...
2024-09-12 13:09:58.565294 Extracting trips ...
Original Dataframe size: 2415636 rows
   Reduced to: 2415636 rows after removing records with speed=0
Creating TrajectoryCollection ...
   Created: TrajectoryCollection with 417 trajectories
Generalizing ...


/home/dragaschnigm/anaconda3/envs/mobiml/lib/python3.10/site-packages/movingpandas/trajectory_generalizer.py:115: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prev_t = temp_df.head(1)["t"][0]
/home/dragaschnigm/anaconda3/envs/mobiml/lib/python3.10/site-packages/movingpandas/trajectory_generalizer.py:115: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prev_t = temp_df.head(1)["t"][0]
/home/dragaschnigm/anaconda3/envs/mobiml/lib/python3.10/site-packages/movingpandas/trajectory_generalizer.py:115: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be

Splitting at observation gaps (1:00:00) ...
   Split: TrajectoryCollection with 861 trajectories
2024-09-12 13:12:19.168622 Computing trajectory features ...
2024-09-12 13:12:19.168643 Enriching trajectories ...
Enriched dataset columns: Index(['traj_id', 'start_t', 'end_t', 'geometry', 'length', 'direction',
       'client', 'mover_id', 'speed_max', 'speed_median', 'H3_seq',
       'speed_start', 'direction_start', 'x_start', 'y_start', 'speed_end',
       'direction_end', 'x_end', 'y_end', 'ship_type'],
      dtype='object')
2024-09-12 13:12:45.082436 training data written to temp/training-data-stationary.pickle


In [20]:
prepare_training_data(mobile_feather_path, mobile_training_file)

2024-09-12 13:12:45.437397 Loading data from temp/ais-vessels.feather ...
2024-09-12 13:12:45.985485 Extracting trips ...
Original Dataframe size: 327510 rows
   Reduced to: 327510 rows after removing records with speed=0
Creating TrajectoryCollection ...
   Created: TrajectoryCollection with 185 trajectories
Generalizing ...


/home/dragaschnigm/anaconda3/envs/mobiml/lib/python3.10/site-packages/movingpandas/trajectory_generalizer.py:115: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prev_t = temp_df.head(1)["t"][0]
/home/dragaschnigm/anaconda3/envs/mobiml/lib/python3.10/site-packages/movingpandas/trajectory_generalizer.py:115: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prev_t = temp_df.head(1)["t"][0]
/home/dragaschnigm/anaconda3/envs/mobiml/lib/python3.10/site-packages/movingpandas/trajectory_generalizer.py:115: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be

Splitting at observation gaps (1:00:00) ...
   Split: TrajectoryCollection with 339 trajectories
2024-09-12 13:13:33.069271 Computing trajectory features ...
2024-09-12 13:13:33.069289 Enriching trajectories ...
Enriched dataset columns: Index(['traj_id', 'start_t', 'end_t', 'geometry', 'length', 'direction',
       'client', 'mover_id', 'speed_max', 'speed_median', 'H3_seq',
       'speed_start', 'direction_start', 'x_start', 'y_start', 'speed_end',
       'direction_end', 'x_end', 'y_end', 'ship_type'],
      dtype='object')
2024-09-12 13:13:41.542096 training data written to temp/training-data-mobile.pickle


# Federated Learning



In [21]:
np.random.seed(0)

vessel_types = ['Cargo', 'Passenger', 'Tanker'] 
n_features = 7
traj_features = ['speed_max', 'speed_median', 'x_start', 'y_start', 'x_end', 'y_end', 'length'] 
test_size = 0.33

trajectory_classifier_model = SummarizedAISTrajectoryClassifier(vessel_types, n_features)


In [22]:
from sklearn.metrics import accuracy_score, confusion_matrix
import json
import os
import numpy as np
from pandas import DataFrame
from mobiml.utils import XYList


def display_confusion_matrix(y_test, predictions, labels):
    cm = confusion_matrix(y_test, predictions, labels=labels)
    cm_df = DataFrame(cm, index=labels, columns=labels)
    print(cm_df)


def save_metrics(predictions, y_test, scenario_name):
    if not os.path.exists("output"):
        os.makedirs("output")

    metrics = {"accuracy": accuracy_score(y_test, predictions)}

    out_path = f"output/fl-global-metrics-{scenario_name}.json"
    print(f"Saving metrics to {out_path}")
    with open(out_path, "w") as fd:
        json.dump(metrics, fd)


def partition(X: np.ndarray, y: np.ndarray, num_partitions: int) -> XYList:
    """Split X and y into a number of partitions."""
    zipped = zip(np.array_split(X, num_partitions), np.array_split(y, num_partitions))
    return list(zipped)

## Define FL client

In [23]:
class AISClient(fl.client.NumPyClient):
    """Client for FL model"""

    def __init__(self, cid, model, partitions, data_loader) -> None:
        super().__init__()
        self.cid = int(cid)
        self.model = model
        
        (X_train, y_train), (self.X_test, self.y_test) = data_loader.load(client_id=self.cid)
        # Split train set into partitions and randomly use one for training.
        partition_id = np.random.choice(partitions)
        (self.X_train, self.y_train) = partition(X_train, y_train, partitions)[partition_id]
        print(f"CLIENT {self.cid} started up, will use partition {partition_id} of {partitions} partitions for training")

    def get_parameters(self, config):  # type: ignore
        return self.model.get_model_parameters()

    def fit(self, parameters, config):  # type: ignore
        self.model.set_model_params(parameters)
        # Ignore convergence failure due to low local epochs       
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            self.model.fit(self.X_train, self.y_train)
            accuracy = self.model.score(self.X_train, self.y_train)

        print(f"CLIENT {self.cid} Training finished for round {config['server_round']}")
        return self.model.get_model_parameters(), len(self.X_train), {"accuracy": accuracy}

    def evaluate(self, parameters, config):  # type: ignore
        self.model.set_model_params(parameters)
        vessel_types = self.model.classes
        loss = log_loss(self.y_test, self.model.predict_proba(self.X_test), labels=vessel_types)
        accuracy = self.model.score(self.X_test, self.y_test)
        print(f"CLIENT {self.cid} accuracy {accuracy}")
        return loss, len(self.X_test), {"accuracy": accuracy}

In [24]:
def generate_client_fn(model, lookup, data_partitions, dloader):
    """ Is called in every training round to initialise a new Client object. """

    def client_fn(cid: str):     
        print(f"******* {int(lookup[cid])} **********")
        return AISClient(int(lookup[cid]), trajectory_classifier_model, data_partitions, dloader).to_client()

    return client_fn

## Start FL 

https://flower.ai/docs/framework/how-to-implement-strategies.html

https://flower.ai/docs/framework/ref-api/flwr.simulation.start_simulation.html

### FL with static data


In [25]:
static_data_loader = AISLoader(vessel_types, traj_features, test_size, path=stationary_training_file)
static_scenario_name = Path(stationary_training_file).stem.replace("training-data-", "")


static_strategy = fl.server.strategy.FedAvg(
        min_available_clients=2,
        evaluate_fn=get_evaluate_fn(trajectory_classifier_model, static_data_loader, static_scenario_name),
        on_fit_config_fn=fit_round,
        evaluate_metrics_aggregation_fn=weighted_average,
        fit_metrics_aggregation_fn=weighted_average,
    )

Vessel types: ['Cargo', 'Passenger', 'Tanker']
Trajectory features: ['speed_max', 'speed_median', 'x_start', 'y_start', 'x_end', 'y_end', 'length']
Test size: 0.33
Filtering ship_type to ['Cargo', 'Passenger', 'Tanker'] ...
... 573 found.
Available trajectory columns: Index(['traj_id', 'start_t', 'end_t', 'geometry', 'length', 'direction',
       'client', 'mover_id', 'speed_max', 'speed_median', 'H3_seq',
       'speed_start', 'direction_start', 'x_start', 'y_start', 'speed_end',
       'direction_end', 'x_end', 'y_end', 'ship_type'],
      dtype='object')
2024-09-12 13:14:29.544573 Splitting dataset ...
Using 209 movers for training and 104 for testing ...
(382 trajectories for training and 191 for testing)


In [26]:
clients_per_round = 3
rounds = 10
client_data_partitions = 2

client_mapping = {  # flwr client id -> MMSI
     '0': 0,
     '1': 1,
     '2': 2,
}


client_fn = generate_client_fn(trajectory_classifier_model, client_mapping, client_data_partitions, static_data_loader)

print(f"{datetime.now()} Starting training")

fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=clients_per_round,
        config=fl.server.ServerConfig(num_rounds=rounds),
        strategy=static_strategy
)
print(f"{datetime.now()} Training done")

INFO :      Starting Flower simulation, config: num_rounds=10, no round_timeout


2024-09-12 13:14:31.227769 Starting training


2024-09-12 13:14:33,480	INFO worker.py:1621 -- Started a local Ray instance.
[2024-09-12 13:14:33,483 I 218203 218203] logging.cc:230: Set ray log level from environment variable RAY_BACKEND_LOG_LEVEL to 3
INFO :      Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'object_store_memory': 6142347264.0, 'memory': 12284694528.0, 'node:192.168.0.241': 1.0, 'CPU': 12.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 12 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(raylet) [2024-09-12 13:14:34,592 I 230696 230696] logging.cc:230: Set ray log level from environment variable RAY_BACKEND_LOG_LEVEL to 3
(raylet) [2024-0

Accuracy 0.39267015706806285
(ClientAppActor pid=230954) ******* 1 **********
(ClientAppActor pid=230954) Client id: 1
(ClientAppActor pid=230954) Vessel types: ['Cargo', 'Passenger', 'Tanker']
(ClientAppActor pid=230954) Trajectory features: ['speed_max', 'speed_median', 'x_start', 'y_start', 'x_end', 'y_end', 'length']
(ClientAppActor pid=230954) Test size: 0.33
(ClientAppActor pid=230954) Filtering ship_type to ['Cargo', 'Passenger', 'Tanker'] ...
(ClientAppActor pid=230954) ... 573 found.
(ClientAppActor pid=230954) Filtering client to 1 ...
(ClientAppActor pid=230954) ... 99 found.
(ClientAppActor pid=230954) Available trajectory columns: Index(['traj_id', 'start_t', 'end_t', 'geometry', 'length', 'direction',
(ClientAppActor pid=230954)        'client', 'mover_id', 'speed_max', 'speed_median', 'H3_seq',
(ClientAppActor pid=230954)        'speed_start', 'direction_start', 'x_start', 'y_start', 'speed_end',
(ClientAppActor pid=230954)        'direction_end', 'x_end', 'y_end', 'ship

INFO :      aggregate_fit: received 2 results and 1 failures
INFO :      fit progress: (1, 1.086115899751846, {'accuracy': 0.39267015706806285}, 2.3014798859985603)
INFO :      configure_evaluate: strategy sampled 3 clients (out of 3)
ERROR :     Traceback (most recent call last):
  File "/home/dragaschnigm/anaconda3/envs/mobiml/lib/python3.10/site-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 73, in _submit_job
    out_mssg, updated_context = self.actor_pool.get_client_result(
  File "/home/dragaschnigm/anaconda3/envs/mobiml/lib/python3.10/site-packages/flwr/simulation/ray_transport/ray_actor.py", line 399, in get_client_result
    return self._fetch_future_result(cid)
  File "/home/dragaschnigm/anaconda3/envs/mobiml/lib/python3.10/site-packages/flwr/simulation/ray_transport/ray_actor.py", line 280, in _fetch_future_result
    res_cid, out_mssg, updated_context = ray.get(
  File "/home/dragaschnigm/.local/lib/python3.10/site-packages/ray/_private/auto_init_hook.py"

...

Accuracy 0.46596858638743455
Accuracy 0.5445026178010471


ERROR :     Traceback (most recent call last):
  File "/home/dragaschnigm/anaconda3/envs/mobiml/lib/python3.10/site-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 73, in _submit_job
    out_mssg, updated_context = self.actor_pool.get_client_result(
  File "/home/dragaschnigm/anaconda3/envs/mobiml/lib/python3.10/site-packages/flwr/simulation/ray_transport/ray_actor.py", line 399, in get_client_result
    return self._fetch_future_result(cid)
  File "/home/dragaschnigm/anaconda3/envs/mobiml/lib/python3.10/site-packages/flwr/simulation/ray_transport/ray_actor.py", line 280, in _fetch_future_result
    res_cid, out_mssg, updated_context = ray.get(
  File "/home/dragaschnigm/.local/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/home/dragaschnigm/.local/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/home/dragas

Accuracy 0.6230366492146597
Accuracy 0.4869109947643979


ERROR :     Traceback (most recent call last):
  File "/home/dragaschnigm/anaconda3/envs/mobiml/lib/python3.10/site-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 73, in _submit_job
    out_mssg, updated_context = self.actor_pool.get_client_result(
  File "/home/dragaschnigm/anaconda3/envs/mobiml/lib/python3.10/site-packages/flwr/simulation/ray_transport/ray_actor.py", line 399, in get_client_result
    return self._fetch_future_result(cid)
  File "/home/dragaschnigm/anaconda3/envs/mobiml/lib/python3.10/site-packages/flwr/simulation/ray_transport/ray_actor.py", line 280, in _fetch_future_result
    res_cid, out_mssg, updated_context = ray.get(
  File "/home/dragaschnigm/.local/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/home/dragaschnigm/.local/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/home/dragas

2024-09-12 13:14:42.100324 Training done
(ClientAppActor pid=230954) CLIENT 1 accuracy 0.0
(ClientAppActor pid=230952) 2024-09-12 13:14:42.051346 Splitting dataset ...
(ClientAppActor pid=230952) Using 209 movers for training and 104 for testing ...
(ClientAppActor pid=230952) (382 trajectories for training and 191 for testing)
(ClientAppActor pid=230952) CLIENT 0 started up, will use partition 0 of 2 partitions for training


### FL with mobile data

In [29]:
mobile_data_loader = AISLoader(vessel_types, traj_features, test_size, path=mobile_training_file)
mobile_scenario_name = Path(mobile_training_file).stem.replace("training-data-", "")


mobile_strategy = fl.server.strategy.FedAvg(
        min_available_clients=2,
        evaluate_fn=get_evaluate_fn(trajectory_classifier_model, mobile_data_loader, mobile_scenario_name),
        on_fit_config_fn=fit_round,
        evaluate_metrics_aggregation_fn=weighted_average,
        fit_metrics_aggregation_fn=weighted_average,
    )

Vessel types: ['Cargo', 'Passenger', 'Tanker']
Trajectory features: ['speed_max', 'speed_median', 'x_start', 'y_start', 'x_end', 'y_end', 'length']
Test size: 0.33
Filtering ship_type to ['Cargo', 'Passenger', 'Tanker'] ...
... 216 found.
Available trajectory columns: Index(['traj_id', 'start_t', 'end_t', 'geometry', 'length', 'direction',
       'client', 'mover_id', 'speed_max', 'speed_median', 'H3_seq',
       'speed_start', 'direction_start', 'x_start', 'y_start', 'speed_end',
       'direction_end', 'x_end', 'y_end', 'ship_type'],
      dtype='object')
2024-09-12 13:15:04.556475 Splitting dataset ...
Using 76 movers for training and 38 for testing ...
(137 trajectories for training and 79 for testing)


In [30]:
clients_per_round = 4
rounds = 10
client_data_partitions = 3


client_mapping = {  # flwr client id -> MMSI
     '0': 236111925,
     '1': 219012959,
     '2': 235662000,
     '3': 265737220,
}


client_fn = generate_client_fn(trajectory_classifier_model, client_mapping, client_data_partitions, mobile_data_loader)

print(f"{datetime.now()} Starting training")
fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=clients_per_round,
        config=fl.server.ServerConfig(num_rounds=rounds),
        strategy=mobile_strategy
)
print(f"{datetime.now()} Training done")

INFO :      Starting Flower simulation, config: num_rounds=10, no round_timeout


2024-09-12 13:15:05.513623 Starting training


2024-09-12 13:15:09,417	INFO worker.py:1621 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'CPU': 12.0, 'node:192.168.0.241': 1.0, 'object_store_memory': 6132514406.0, 'memory': 12265028814.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 12 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(raylet) [2024-09-12 13:15:10,601 I 231846 231846] logging.cc:230: Set ray log level from environment variable RAY_BACKEND_LOG_LEVEL to 3
(raylet) [2024-09-12 13:15:11,493 I 232109 232109] logging.cc:230: Set ray log level from environment variable RAY_BACKEND_LOG_LEVEL to 3
(raylet

(ClientAppActor pid=232117) ******* 235662000 **********
(ClientAppActor pid=232117) Client id: 235662000
(ClientAppActor pid=232117) Vessel types: ['Cargo', 'Passenger', 'Tanker']
(ClientAppActor pid=232117) Trajectory features: ['speed_max', 'speed_median', 'x_start', 'y_start', 'x_end', 'y_end', 'length']
(ClientAppActor pid=232117) Test size: 0.33
(ClientAppActor pid=232117) Filtering ship_type to ['Cargo', 'Passenger', 'Tanker'] ...
(ClientAppActor pid=232117) ... 216 found.
(ClientAppActor pid=232117) Filtering client to 235662000 ...
(ClientAppActor pid=232117) ... 0 found.
(ClientAppActor pid=232117) Available trajectory columns: Index(['traj_id', 'start_t', 'end_t', 'geometry', 'length', 'direction',
(ClientAppActor pid=232117)        'client', 'mover_id', 'speed_max', 'speed_median', 'H3_seq',
(ClientAppActor pid=232117)        'speed_start', 'direction_start', 'x_start', 'y_start', 'speed_end',
(ClientAppActor pid=232117)        'direction_end', 'x_end', 'y_end', 'ship_type'

RuntimeError: Simulation crashed.